# Batch Normalization

## Introdução

Batch Normalization é uma técnica que permite normalizar automaticamente os valores que atravessam uma camada da rede neural. Normalmente o uso do BatchNorm permite um treinamento mais rápido e com menores cuidados na inicialização dos pesos. Permite que redes profundas sejam treinadas mais facilmente. 

A normalização é feita para que o resultado tenha média zero e variância unitária, porém, em seguida, o resultado é
escalado pelo fator $\gamma$ e somado ao fator $\beta$. Estes dois fatores são parâmetros que serão também otimizados durante o treinamento do gradiente descendente.

Atualmente a técnica de batch normalization é utilizada em todas as redes profundas. 

A camada de batch normalization é colocada entre a camada densa ou convolucional e antes da camada de ativação.

**Referência:**
- Ioffe, S. and Szegedy, C. (2015), Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift., in Francis R. Bach & David M. Blei, ed., 'ICML' , JMLR.org, , pp. 448-456. 
    [PDF:arxiv](http://arxiv.org/pdf/1502.03167.pdf) 

<img src='../figures/batchnorm_neuronios.png', width=900></img>

<img src='../figures/batchnorm_equations.png',width=500></a>

## Diferença entre fase de treinamento e fase de predição ou avaliação

A normalização ocorre de forma distinta na fase de treinamento e na fase de avaliação:
- em treinamento: a média e variância ($\mu$ e $\sigma^2$) são estimados a partir dos valores
das amostras no mini-batch:
    - `running_mean`
    - `running_var`
- em avaliação: a média e variância é calculada pela média móvel, definida pelo parâmetro momentum ($\lambda$):

\begin{align*} 
\boldsymbol{m}_{t} &=  \lambda \boldsymbol{m}_{t-1} + (1 - \lambda) \boldsymbol{\mu_t} &&
\boldsymbol{v}_{t} =  \lambda \boldsymbol{v}_{t-1} + (1 - \lambda) \boldsymbol{\sigma^{2}_t}
\end{align*}

## Importações

In [13]:
%matplotlib inline
import matplotlib.pyplot as plot
from IPython import display

import sys
import numpy as np
import numpy.random as nr
import pandas as pd

import torch
from torch.autograd import Variable

np.set_printoptions(precision=2, suppress=True)
nr.seed(23456)


## Batch Norm: parâmetros

### nn.BatchNorm1d - Entrada com dimensão 2

- Documentação oficial: http://pytorch.org/docs/master/nn.html#normalization-layers

Quando a entrada da camada tem duas dimensões (amostras e atributos) a normalização por atributo. Ou seja, calcula-se a estatística (média e variância) para cada coluna da matriz de dados, em cada *mini-batch*.

## Experimento 1 - aplicação BatchNorm tanto para treino como para teste

### Criação da camada BatchNorm

In [184]:
torch.manual_seed(1234)

model = torch.nn.BatchNorm1d(5)        # Rede formada de apenas uma camada BatchNorm

x = Variable(torch.randn(4, 5)) * 100. # Entrada x
model.training = True

y = model(x)                           # Forward

### Entrada x

In [185]:
x

Variable containing:
-133.6756 -113.6071   23.2839 -172.4477   51.1140
 -61.2595   47.0261   65.2737   57.1934  170.6041
 100.2897 -231.1064  110.7505  171.6769   70.1500
  62.7425  117.5225    9.5521 -151.1595   12.5337
[torch.FloatTensor of size 4x5]

### Parâmetros: weight e bias -> gamma e beta

In [186]:
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

weight
[ 0.19  0.5   0.62  0.82  0.44]
bias
[ 0.  0.  0.  0.  0.]
running_mean
[-0.8  -4.5   5.22 -2.37  7.61]
running_var
[ 1179.64  2475.06   209.37  2770.49   455.28]


### Saída  y

In [187]:
y

Variable containing:
-0.2560 -0.2505 -0.4552 -0.8442 -0.1874
-0.1085  0.3364  0.2055  0.4589  0.7086
 0.2205 -0.6798  0.9209  1.1086 -0.0446
 0.1440  0.5939 -0.6712 -0.7234 -0.4766
[torch.FloatTensor of size 4x5]

### Conferindo com código próprio

$$  y = \frac{x - mean[x]}{ \sqrt{Var[x] + \epsilon}} * gamma + beta $$

In [188]:
gamma, beta, mv_mean, mv_var = model.state_dict().values()

if model.training:
    mean = x.mean(dim=0).data
    var  = x.var(dim=0, unbiased=False).data
else:
    mean = mv_mean
    var  = mv_var

y2 = (x.data - mean) / torch.sqrt(var + model.eps) * gamma + beta

print(y2)


-0.2560 -0.2505 -0.4552 -0.8442 -0.1874
-0.1085  0.3364  0.2055  0.4589  0.7086
 0.2205 -0.6798  0.9209  1.1086 -0.0446
 0.1440  0.5939 -0.6712 -0.7234 -0.4766
[torch.FloatTensor of size 4x5]



## Experimento 2 - Verificando o cálculo das médias móveis

In [234]:
torch.manual_seed(1234)
momentum = 0.1

model = torch.nn.Sequential(
        torch.nn.BatchNorm1d(5, momentum=momentum),
        torch.nn.Linear(5, 1),
        torch.nn.Sigmoid()
        )
model
#model = Sequential()
#model.add(BatchNormalization(axis=1, input_shape=(5,), momentum=mom, epsilon=0.0001))
#model.add(Dense(1, activation='sigmoid'))

Sequential (
  (0): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True)
  (1): Linear (5 -> 1)
  (2): Sigmoid ()
)

In [235]:
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

0.weight
[ 0.19  0.5   0.62  0.82  0.44]
0.bias
[ 0.  0.  0.  0.  0.]
0.running_mean
[ 0.  0.  0.  0.  0.]
0.running_var
[ 1.  1.  1.  1.  1.]
1.weight
[[ 0.1   0.26  0.24  0.25  0.32]]
1.bias
[-0.2]


In [240]:
#print([x.shape for x in model.get_weights()], end='\n\n')
#model.set_weights([[ 1., 1., 1., 1., 1.],     # gamma
#                   [ 0., 0., 0., 0., 0.],     # beta
#                   [ 0., 0., 0., 0., 0.],     # moving_mean
#                   [ 1., 1., 1., 1., 1.]])    # moving_variance

gamma, beta, mv_mean, mv_var, w, b = model.state_dict().values()

x = torch.randn(4, 5) * 20.
y = torch.randn(4, 1)
print(x)
print('mean do minibatch:',x.mean(0).numpy())
print('var  do minibatch: ',x.var(0,unbiased=False).numpy())


 -2.3211  -5.7813  27.0910 -12.8426  23.3973
-13.2257  24.3948  -3.8164  -4.7489   3.5889
 -2.0350  25.5979 -16.1929   7.6010   2.1562
-23.5506  -7.8922  -5.7136  23.8806  13.3141
[torch.FloatTensor of size 4x5]

mean do minibatch: [-10.28   9.08   0.34   3.47  10.61]
var  do minibatch:  [  79.03  254.07  260.72  191.83   72.9 ]


In [241]:
mv_mean_new = (1 - momentum) * x.mean(0) + momentum * mv_mean
mv_var_new  = (1 - momentum) * x.var(0,unbiased=False) + momentum * mv_var
print('my running mean:',mv_mean_new.numpy())
print('my running var :',mv_var_new.numpy())

my running mean: [-9.25  8.17  0.31  3.13  9.55]
my running var : [  71.22  228.77  234.75  172.74   65.71]


In [242]:
from lib.pytorch_utils import DeepNetTrainer
from torch.utils.data import TensorDataset, DataLoader

net_trainer = DeepNetTrainer(model=model,
                             optimizer = torch.optim.SGD(model.parameters(),lr=0.1),
                             criterion = torch.nn.MSELoss()
                            )
train_data = DataLoader(TensorDataset(x,y),batch_size=4)
net_trainer.fit(1, train_data,use_gpu=False)

Starting training for 1 epochs

  1:   0.0s   T: 0.92809 best

Model from epoch 1 saved as "None.*", loss = 0.92809


In [243]:
param_dict = model.state_dict()
for name,value in param_dict.items():
    print(name)
    print(value.numpy())

0.weight
[ 0.19  0.5   0.62  0.82  0.44]
0.bias
[-0.   -0.01 -0.01 -0.01 -0.01]
0.running_mean
[-1.03  0.91  0.03  0.35  1.06]
0.running_var
[ 11.44  34.78  35.66  26.48  10.62]
1.weight
[[ 0.1   0.25  0.24  0.26  0.32]]
1.bias
[-0.24]


# A partir daqui era o código do Keras que pode ser retirado depois de conseguirmos fazer a parte de cima funcionar

In [59]:

sgd = SGD(0.5)
model.compile(sgd, loss='mse')
h = model.fit(x, y, batch_size=10, epochs=1, verbose=0)

new_weights = model.get_weights()

print('lambda:', new_weights[0])
print('beta:  ', new_weights[1])
print()
print('mv_mean:', new_weights[2])
print('        ', mv_mean_new)
print('mv_var: ', new_weights[3])
print('        ', mv_var_new)

[(5,), (5,), (5,), (5,), (5, 1), (1,)]

[[  1.74   4.4    7.11  15.93   7.74]
 [ 15.79  16.52   3.59   8.45   0.57]
 [  6.19   1.2   19.06   9.33   6.74]
 [ 14.86   8.33  19.11  12.7   10.43]
 [  6.52  11.71   0.5   11.32  17.42]
 [  2.81   6.7   15.46  13.86  18.72]
 [ 19.9    5.29  15.4    6.41   5.52]
 [ 15.81   4.68   0.07   0.84  11.68]
 [ 13.75  17.97  17.19   6.23  12.38]
 [  1.05  18.93   1.34   6.03   2.17]] [ 9.84  9.57  9.88  9.11  9.34] [ 6.55  6.    7.67  4.25  5.67]

lambda: [ 1.    0.98  0.98  1.    1.  ]
beta:   [-0.03 -0.02  0.03  0.   -0.02]

mv_mean: [ 7.87  7.66  7.91  7.29  7.47]
         [ 7.87  7.66  7.91  7.29  7.47]
mv_var:  [ 34.54  28.98  47.32  14.66  25.91]
         [ 34.54  28.98  47.32  14.66  25.91]
